In [16]:
# !pip3 install transformers
# !pip3 install tensorflow
# !pip3 install torch
# !pip3 install weaviate

## Import the BERT transformer model and pytorch

We are using the `bert-base-uncased` model in this example, but any model will work. Feel free to adjust accordingly.

In [25]:
import torch
from transformers import AutoModel, AutoTokenizer, BertTokenizer


torch.set_grad_enabled(False)

# udpated to use different model if desired
MODEL_NAME = "bert-base-uncased"

# Create model and tokenizer
model = AutoModel.from_pretrained(MODEL_NAME)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

## Initialize Weaviate Client
This assumes you have Weaviate running locally on `:8080`. Adjust URL accordingly. You could also enter the WCS URL here, for example, if you are running a WCS cloud instance instead of running Weaviate locally.

In [42]:
import weaviate

client = weaviate.Client("http://localhost:8080")

## Load dataset from disk
Create some helper functions to create the dataset (20-newsgroup text posts) from disk. These methods are specific to the structure of your dataset, adjust accordingly.

In [26]:
import os
import random

def get_post_filenames(limit_objects=100):
    file_names = []
    i=0
    for root, dirs, files in os.walk("./data/20news-bydate-test"):
        for filename in files:
            path = os.path.join(root, filename)
            file_names += [path]
        
    random.shuffle(file_names)
    limit_objects = min(len(file_names), limit_objects)
      
    file_names = file_names[:limit_objects]

    return file_names

def read_posts(filenames=[]):
    posts = []
    for filename in filenames:
        f = open(filename)
        ## TODO: strip headers
        posts += [f.read()]

    return posts       


## Vectorize Dataset using BERT

The following is a helper function to vectorize all posts (using our BERT transformer) which are entered as an array. The return array contains all the vectors in the same order. BERT is optimized to run on GPUs, if you're using CPUs this might take a while. 

In [47]:
import time

def vectorize_posts(posts=[]):
    print("Vectorize your posts with BERT. If you are using CPUs this might take a while...")
    post_vectors=[]
    before=time.time()
    for post in posts:
        tokens_pt = tokenizer(post, padding=True, truncation=True, max_length=500, add_special_tokens = True, return_tensors="pt")
        outputs = model(**tokens_pt)
        post_vectors += outputs.pooler_output
    after=time.time()
    
    print("Vectorized {} items in {}s".format(len(posts), after-before))
    
    return post_vectors

### Run everything we have so far

It is now time to run the functions we defined before. Let's load 50 random posts from disk, then vectorize them using BERT.

In [49]:
posts = read_posts(get_post_filenames(50))
vectors = vectorize_posts(posts)

Vectorize your posts with BERT. If you are using CPUs this might take a while...
Vectorized 50 items in 41.9816780090332s


## Initialize Weaviate

Now that we have vectors we can import both the posts and the vectors into Weaviate, so we can then search through them.

### Init a simple schema
Our schema is very simple, we just have one object class, the "Post". A post class has just a single property, which we call "content" and is of type "text".

Each class in schema creates one index, so by running the below we tell weaviate to create one brand new vector index waiting for us to import data.

In [46]:
def init_weaviate_schema(client):
    
    # a simple schema containing just a single class for our posts
    schema = {
        "classes": [{
                "class": "Post",
                "vectorizer": "none", # explicitly tell Weaviate not to vectorize anything, we are providing the vectors ourselves through our BERT model
                "properties": [{
                    "name": "content",
                    "dataType": ["text"],
                }]
        }]
    }
    
    # cleanup from previous runs
    client.schema.delete_all()
    
    client.schema.create(schema)

In [ ]:
init_weaviate_schema(client)

In [ ]:
def import_posts_with_vectors(posts, vectors):
    